### Create a Lakehouse


1. upload: https://raw.githubusercontent.com/MicrosoftLearning/dp-data/main/sales.csv
2. Create a shortcut
3. Load data file into table
4. SQL Query for Revenue

In [ ]:
SELECT Item, SUM(Quantity * UnitPrice) AS Revenue
FROM sales
GROUP BY Item
ORDER BY Revenue DESC;

5. Visual Query LineItems per each Sales order
6. Create a report "Items sold"


### Ingest Data from Pipeline


1. Connect Data source  
    **File format:** DelimitedText  
    **Column delimiter**: Comma (,)  
    **Row delimiter:** Line feed (\n)  
    **First row as header:** Selected
2. Create Notebook to rename table   
    table_name = "sales"
3. Toggle parameters
4. Transform ingested table

### Analyze Data with Apache Spark

Read Line 1 as data

In [ ]:
table_name = "name"

In [ ]:


df = spark.read.format("csv").option("header","true").load("Files/data/*.csv")

display(df)


Filter 

In [ ]:
customers = df.select("CustomerName", "Email").where(df['Item']=='Road-250 Red, 52')
print(customers.count())
print(customers.distinct().count())

display(customers.distinct())

Aggregate and Group Data

In [ ]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()

display(productSales)

In [ ]:
from pyspark.sql.functions import *

yearlySales = df.select(year(col("OrderDate")).alias("Year")).groupBy("Year").count().orderBy("Year")

display(yearlySales)

Transform and Save

In [ ]:
## Add month and year columns
df = df.withColumn("Year", year(col("OrderDate"))).withColumn("Month", month(col("OrderDate")))

# Derive FirstName and LastName columns
df = df.withColumn("FirstName", split(col("CustomerName"), " ").getItem(0)).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))

# Filter and reorder columns
df = df["SalesOrderNumber", "SalesOrderLineNumber", "OrderDate", "Year", "Month", "FirstName", "LastName", "EmailAddress", "Item", "Quantity", "UnitPrice", "TaxAmount"]

# Load the data into a table
df.write.format("delta").mode("append").saveAsTable(table_name)